# Introduction
Now that I have removed the RNA/DNA node and we have fixed many pathways, I will re-visit the things that were raised in issue #37: 'Reaction reversibility'. There were reactions that we couldn't reverse or remove or they would kill the biomass. I will try to see if these problems have been resolved now. If not, I will dig into the underlying cause in a smanner similar to what was done in notebook 20. 

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction

In [2]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [3]:
model_e_coli = cameo.load_model('iML1515')

In [4]:
model_b_sub = cameo.load_model('iYO844')

__ALDD2x__
should be irreversible, but doing so kills the biomass growth completely at this moment. It needs to be changed as we right now have an erroneous energy generating cycle going from aad_c --> ac_c (+atp) --> acald --> accoa_c -->aad_c.
Apparently, unconciously i already fixed this problem in notebook 20. So this is fine now. 

__GLYO1__ This reaction has already been removed in notebook 20 to fix the glycine pathway.

__DHORDfum__ Has been renamed to DHORD6 in notebook 20 in the first check of fixing dCMP. And the reversability has been fixed too. 

__OMPDC__ This has by chance also been fixed in notebook 20 in the first pass to fix dCMP biosynthesis.

__NADK__ The reaction is currently reversible, but should be irreversible, producing nadp and adp. 
Still, when I try to fix the flux in the direction it should be, it kills the biomass production. I will try to figure out why, likely it has to do with co-factor balance.

In [6]:
model.reactions.NADK.bounds = (0,1000)

In [7]:
model.reactions.ALAD_L.bounds = (-1000,0)

In [8]:
model.optimize().objective_value

0.0

In [9]:
cofactors = ['nad_c', 'nadh_c','', '', '', '']

with model:
#     model.add_boundary(model.metabolites.glc__D_c, type = 'sink', reaction_id = 'test')
#     model.add_boundary(model.metabolites.r5p_c , type = 'sink', reaction_id = 'test2')
#     model.add_boundary(model.metabolites.hco3_c, type = 'sink', reaction_id = 'test3')
    for met in model.reactions.biomass.metabolites:
        if met.id in cofactors:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff}) 
        else: 
            continue
    solution = model.optimize()
    #print (model.metabolites.glu__D_c.summary())
    #print ('test flux:', solution['test'])
    #print ('test2 flux:', solution['test2'])
    print (solution.objective_value)

1.8496633304871162


It seems that the NAD and NADH are the blocked metabolites for biomass generation. Now lets try to figure out where this problem lies. 
I think the problem lies in re-generating NAD. The model uses this reaction togehter with oth strange reactions to regenerate NAD, where normally in oxygen containing conditions I would expect respiration to do this. So let me see how bacillus and e. coli models do this and see if maybe some form of ETC is missing in our model. This would explain why adding the ATP synthase didn't influence our biomass prediction at all.

__Flavin reductase__
In E. coli we observed that there is a flavin reductase in the genome, contributing to NADH regeneration. We've looked into the genome annotation for our strain, and have found that there is a flavin reductase annotated there aswell (https://www.genome.jp/dbget-bin/www_bget?ptl:AOT13_02085), but not in bacillus (fitting the model). Therefore, I will add this reaction into our model, named FADRx. 

__NADH dehydrogenase__
The NADH dehydrogenase, tansfering reducing equivalents from NADH to quinone, is the first part of the electron transport chain. The quinones then can transfer the electrons to pump out protons, which can allow ATP synthase to generate additional energy. in iML1515 this reaction is captures by NADH16pp, NADH17pp and NADH18pp. In B. subtilis NADH4 reflects this reaction. In our model, we don't currently have anything that resembles this reaction. However, in Beata's thesis (and the genome) we can find EC 1.6.5.3, which performs the a similar reaction to NADH16pp. Therefore, I will add this reactin into our model.

In our model, we also have the reactions QH2OR and NADHQOR, which somewhat resemble the NADHDH reaction. Both do not include proton translocation or are reversible. To prevent these reactions from forming a cycle and having incorrect duplicate reactions in the model, I will remove them. 

__CYOR__
The last 'step' in the model electron transport chain is the transfer of electrons from the quinone to oxygen, pumping protons out of the cell. E. coli has a CYTBO3_4pp reaction that shows this, performed by a cytochrome oxidase. The model doesnt have this reaction, but from Beata's thesis and the genome annotation one would expect this to be present. We found the reaction in a way similar to the E. coli model. Therefor I will add the CYTBO3 reaction to our model, as indicated in Beata's thesis. 


In [10]:
model.add_reaction(Reaction(id='FADRx'))

In [11]:
model.reactions.FADRx.name = 'Flavin reductase'

In [12]:
model.reactions.FADRx.annotation = model_e_coli.reactions.FADRx.annotation

In [13]:
model.reactions.FADRx.add_metabolites({
    model.metabolites.fad_c:-1, 
    model.metabolites.h_c: -1, 
    model.metabolites.nadh_c:-1,
    model.metabolites.fadh2_c:1, 
    model.metabolites.nad_c:1
})

In [14]:
#add NADH dehydrogenase reaction
model.add_reaction(Reaction(id='NADHDH'))

In [15]:
model.reactions.NADHDH.name = 'NADH Dehydrogenase (ubiquinone & 3.5 protons)'

In [16]:
model.reactions.NADHDH.annotation['ec-code'] = '1.6.5.3'
model.reactions.NADHDH.annotation['kegg.reaction'] = 'R11945'

In [17]:
model.reactions.NADHDH.add_metabolites({
    model.metabolites.nadh_c:-1, model.metabolites.h_c: -4.5, model.metabolites.ubiquin_c:-1,
    model.metabolites.nad_c: 1, model.metabolites.h_e: 3.5, model.metabolites.qh2_c: 1
})

In [18]:
model.remove_reactions(model.reactions.NADHQOR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list



In [19]:
model.remove_reactions(model.reactions.QH2OR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [20]:
model.add_reaction(Reaction(id='CYTBO3'))

In [21]:
model.reactions.CYTBO3.name = 'Cytochrome oxidase bo3 (ubiquinol: 2.5 protons)'

In [22]:
model.reactions.CYTBO3.add_metabolites({
    model.metabolites.o2_c:-0.5, model.metabolites.h_c: -2.5, model.metabolites.qh2_c:-1, 
    model.metabolites.h2o_c:1, model.metabolites.h_e: 2.5, model.metabolites.ubiquin_c:1
})

In looking at the above, I also observed some other reactions that probably should not looked at and modified.

In [23]:
model.reactions.MALQOR.id = 'MDH2'

In [24]:
model.reactions.MDH2.bounds = (0,1000)

In [25]:
model.metabolites.hexcoa_c.id = 'hxcoa_c'

In [26]:
model.reactions.HEXOT.id = 'ACOAD2f'

In [27]:
model.metabolites.dccoa_c.id = 'dcacoa_c'

In [28]:
model.reactions.DECOT.id = 'ACOAD4f'

In [29]:
#in the wrong direction and id
model.reactions.GLYCDH_1.id = 'HPYRRx'

In [30]:
model.reactions.HPYRRx.bounds = (-1000,0)

In [31]:
#in the wrong direction
model.reactions.FMNRx.bounds = (-1000,0)

In [32]:
model.metabolites.get_by_id('3hbycoa_c').id = '3hbcoa_c'

Even with the changes above we still do not restore growth... Supplying nmn_c restores growth, but supplying aspartate (beginning of the pathway) doesn't sovle the problem. so maybe the problem lies more with the NAD biosynthesis pathway than really the regeneration anymore?

In [33]:
model.metabolites.nicrnt_c.name = 'Nicotinate ribonucleotide'

In [34]:
model.metabolites.ncam_c.name = 'Niacinamide'

In [35]:
#wrong direction
model.reactions.QULNS.bounds = (-1000,0)
#this rescued biomass accumulation! 

In [36]:
#connected to aspartate

In [41]:
model.optimize().objective_value

1.7838848823121336

In [37]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

Flux is carried through the 
Still it is strange that flux is not carried through the ETC, but is through the ATP synthase as one would expect in the presence of oxygen. Therefore I will investigate where the extracellular protons come from. 

It seems all the extracellular protons come from the export of phosphate (pi_c) which is proton symport coupled. We are producing so much phosphate from the biomass reaction. Though in theory, phosphate should not be produced so much, as it is also used for the generation of ATP from ADP. Right now I don't really see how to solve this problem. I've made an issueof it and will look into this at another time. 

In [45]:
model.optimize()['ATPS4r']

40.97033572180817

In [54]:
model.metabolites.pi_c.summary()

I also noticed that now most ATP comes from dGTP. The production of dGDP should just play a role in supplying nucleotides for biomass and so the flux it carries be low. I will check where the majority of the dGTP comes from.

What is happening is the following: dgtp is converted dgdp and atp (rct ATDGDm). The dgdp then reacts with pep to form dGTP again. Pep formation is somewhat energy neutral, but it is wierd the metabolism decides to do this by themselves instead of flowing the pep into pyruvate via the normal glycolysis and into the TCA.

In [80]:
#reaction to be removed
model.remove_reactions(model.reactions.PYRPT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



Removing these reactions triggers normal ATP production via ETC and ATP synthase again. So this may be solved now. 

In [85]:
model.metabolites.pi_c.summary()

In [86]:
#save & commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')